<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>COMP 555 Bioalgorithms Spring 2021</h1>
<h1 style="font-size: 200%;">Problem Set 4</h1>
</div>

---
The following problems make use of the following Insulin protein sequences:

In [1]:
human = 'MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYCN'
dog   = 'MALWMRLLPLLALLALWAPAPTRAFVNQHLCGSHLVEALYLVCGERGFFYTPKARREVEDLQVRDVELAGAPGEGGLQPLALEGALQKRGIVEQCCTSICSLYQLENYCN'

---
**Problem 1:** Find the longest common subsequence (LCS) shared the between human and dog insulin, its length, and the alignment that it implies.

In [2]:
import numpy as np

def findLCS(v, w):
    score = np.zeros((len(v)+1, len(w)+1), dtype='int32')
    backt = np.zeros((len(v)+1, len(w)+1), dtype='int32')
    for i in range(1, len(v)+1):
        for j in range(1, len(w)+1):
            if v[i-1] == w[j-1]:
                score[i, j], backt[i, j] = max((score[i-1, j-1]+1, 3), (score[i-1, j], 1), (score[i, j-1], 2))
            else:
                score[i, j], backt[i, j] = max((score[i-1, j], 1), (score[i, j-1], 2))
    return score, backt

def LCS(b, v, i, j):
    if i == 0 and j == 0:
        return ''
    elif b[i, j] == 3:
        return LCS(b, v, i-1, j-1) + v[i-1]
    elif b[i, j] == 2:
        return LCS(b, v, i, j-1)
    else:
        return LCS(b, v, i-1, j)

def Alignment(b, v, w, i, j):
    if i == 0 and j == 0:
        return ['', '']
    if b[i, j] == 3:
        result = Alignment(b, v, w, i-1, j-1)
        result[0] += v[i-1]
        result[1] += w[j-1]
        return result
    if b[i, j] == 2:
        result = Alignment(b, v, w, i, j-1)
        result[0] += '_'
        result[1] += w[j-1]
        return result
    if b[i, j] == 1:
        result = Alignment(b, v, w, i-1, j)
        result[0] += v[i-1]
        result[1] += '_'
        return result

In [3]:
s, b = findLCS(human, dog)
lcs = LCS(b, human, b.shape[0]-1, b.shape[1]-1)
print(lcs)
print(len(lcs))
print()
align = Alignment(b, human, dog, b.shape[0]-1, b.shape[1]-1)
h_align_p1 = align[0]
d_align_p1 = align[1]

print('h:', h_align_p1)
print('d:', d_align_p1)

MALWMRLLPLLALLALWAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKRREEDLQGLQPLALEGLQKRGIVEQCCTSICSLYQLENYCN
90

h: MALWMRLLPLLALLALWGPDPA_A___AFVNQHLCGSHLVEALYLVCGERGFFYTPKT_RREA_EDLQ_______GS______LQPLALEGS_LQKRGIVEQCCTSICSLYQLENYCN
d: MALWMRLLPLLALLALW____APAPTRAFVNQHLCGSHLVEALYLVCGERGFFYTPK_ARRE_VEDLQVRDVELAG_APGEGGLQPLALEG_ALQKRGIVEQCCTSICSLYQLENYCN


---
**Problem 2:** Find the best *global alignment* between the human and dog insulin assuming the following scoring from the **Alignment Game** described in lecture:

* +1 for a match
* 0 for a mismatch
* -1 for an indel

What is score for this alignment and the alignment itself?

In [4]:
def GlobalAlignment(v, w, indel):
    s = np.zeros((len(v)+1, len(w)+1), dtype='int32')
    b = np.zeros((len(v)+1, len(w)+1), dtype='int32')
    for i in range(0, len(v)+1):
        for j in range(0, len(w)+1):
            if j == 0:
                if i > 0:
                    s[i, j] = s[i-1, j] + indel
                    b[i, j] = 1
                continue
            if i == 0:
                s[i, j] = s[i, j-1] + indel
                b[i, j] = 2
                continue
            score = s[i-1, j-1] + (v[i-1] == w[j-1])
            vskip = s[i-1, j] + indel
            wskip = s[i, j-1] + indel
            s[i,j] = max(vskip, wskip, score)
            if s[i, j] == vskip:
                b[i, j] = 1
            elif s[i, j] == wskip:
                b[i, j] = 2
            else:
                b[i, j] = 3
    return (s, b)

In [5]:
s, b = GlobalAlignment(human, dog, -1)
print('Best score =', s[-1, -1])
print()
align = Alignment(b, human, dog, b.shape[0]-1, b.shape[1]-1)
h_align_p2 = align[0]
d_align_p2 = align[1]

print('h:', h_align_p2)
print('d:', d_align_p2)

Best score = 78

h: MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQ_______GS_____LQPLALEGSLQKRGIVEQCCTSICSLYQLENYCN
d: MALWMRLLPLLALLALWAPAPTRAFVNQHLCGSHLVEALYLVCGERGFFYTPKARREVEDLQVRDVELAGAPGEGGLQPLALEGALQKRGIVEQCCTSICSLYQLENYCN


---
**Problem 3:** According to the rules of the **Alignment Game**, what score does your alignment from **Problem 1** achieve?

In [6]:
ha, da = h_align_p1, d_align_p1
score = sum([1 if ha[i] == da[i] else -1 if ha[i] == '_' or da[i] == '_' else 0 for i in range(len(ha))])
print(score)

62


---
**Problem 4:** Find the longest *embedded* common subsequence in the human insulin sequence.

The longest embedded common subsequence is the longest subsequence that appears twice in a single given sequence for which no symbol matches itself. As an example, consider <code>TAGACTAAG</code>. The longest common subsequence is the sequence itself, <code>TAGACTAAG</code>, but the longest *embedded* common subsequence is <code>TAG</code>. 

In [7]:
def findLECS(v):
    score = np.zeros((len(v)+1, len(v)+1), dtype='int32')
    backt = np.zeros((len(v)+1, len(v)+1), dtype='int32')
    for i in range(1, len(v)+1):
        for j in range(1, len(v)+1):
            if v[i-1] == v[j-1] and i != j:
                score[i, j], backt[i, j] = max((score[i-1, j-1]+1, 3), (score[i-1, j], 1), (score[i, j-1], 2))
            else:
                score[i, j], backt[i, j] = max((score[i-1, j], 1), (score[i, j-1], 2))
    return score, backt

def LECS(b, v, i, j):
    if i == 0 and j == 0:
        return ''
    elif b[i, j] == 3:
        return LECS(b, v, i-1, j-1) + v[i-1]
    elif b[i, j] == 2:
        return LECS(b, v, i, j-1)
    else:
        return LECS(b, v, i-1, j)

In [8]:
s, b = findLECS(human)
lecs = LECS(b, human, b.shape[0]-1, b.shape[1]-1)
print(lecs)

MLLLLLALALGAVCGEGFTREAEGSLQESICSLYN


In [9]:
# Testing for correctness
def AlignmentLECS(b, v, w, i, j):
    if b[i, j] == 3:
        result = AlignmentLECS(b, v, w, i-1, j-1)
        result[0] += v[i-1]
        result[1] += w[j-1]
        return result
    if b[i, j] == 2:
        result = AlignmentLECS(b, v, w, i, j-1)
        result[0] += '_'
        result[1] += w[j-1]
        return result
    if b[i, j] == 1:
        result = AlignmentLECS(b, v, w, i-1, j)
        result[0] += v[i-1]
        result[1] += '_'
        return result
    if b[i, j] == 0:
        return ['', '']

aa, ab = AlignmentLECS(b, human, human, len(human), len(human))
print(aa)
print(ab)

MRL_LP___LLA_LLALWGPDPAA_AFVNQHLC_GSHLVE___ALYL___V____CG____ER______GF___FYTPK__TR___REA_EDLQGSLQPLALE___GSLQKRGIV____EQCCT_S_____I____C_SLYQ___LENYC___N___
M__AL_WMRLL_PLLAL_______LA_____L_WG_____PDPA___AAFVNQHLCGSHLVE_ALYLVCG_ERGF____FYT_PKTR__RE_________A_EDLQGSLQ_____PLALE____GSLQKRGIVEQCCTS___ICSL__Y_QLENYCN


In [10]:
checker = ''
for i in range(len(aa)):
    if aa[i] == ab[i]:
        checker += aa[i]
print(checker)

MLLLLLALALGAVCGEGFTREAEGSLQESICSLYN


---
**Problem 5:** Output the longest *embedded* common subsequence from the string given in **Problem 4** in the following comma separated format:

    positionInFirstSubsequence, positionInSecondSubsequence, commonSymbol

with one symbol per line. For <code>TAGACTAAG</code> the output could be:

    0, 5, T
    1, 7, A
    2, 8, G

In [11]:
def LECS(b, v, i, j):
    if i == 0 and j == 0:
        return []
    elif b[i, j] == 3:
        return LECS(b, v, i-1, j-1) + [(j-1, i-1, v[i-1])]
    elif b[i, j] == 2:
        return LECS(b, v, i, j-1)
    else:
        return LECS(b, v, i-1, j)

In [12]:
s, b = findLECS(human)
lecs = LECS(b, human, b.shape[0]-1, b.shape[1]-1)

# for t in lecs:
#     print('%2d, %2d, %s' % t)

for a, b, c in lecs:
    print('%2d, %2d, %s' % (a, b, c))

 0,  4, M
 2,  7, L
 6,  9, L
 7, 10, L
 9, 12, L
10, 13, L
11, 14, A
12, 15, L
14, 23, A
15, 29, L
17, 31, G
21, 37, A
25, 41, V
30, 42, C
31, 43, G
36, 44, E
43, 46, G
47, 48, F
50, 53, T
54, 55, R
56, 58, E
57, 68, A
58, 70, E
62, 71, G
63, 72, S
64, 73, L
65, 74, Q
70, 80, E
72, 85, S
78, 86, I
83, 87, C
85, 88, S
89, 92, L
90, 95, Y
94, 97, N
